<a href="https://colab.research.google.com/github/EderCabascango/Notebooks-Ciencia-de-datos/blob/main/Azure_Lanzar_Experimento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print("Workspace:", ws.name)
print("Location:", ws.location)
print("Resource Group:", ws.resource_group)
print("Subscription:", ws.subscription_id)


In [ ]:
train_py = r"""
import os
import joblib
import mlflow
import mlflow.sklearn

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

def main():
    # Dataset ejemplo
    X, y = load_iris(return_X_y=True)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    # Entrenamiento
    model = LogisticRegression(max_iter=300)
    model.fit(X_train, y_train)

    # Evaluación
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)

    # MLflow tracking (métricas)
    mlflow.log_metric("accuracy", acc)

    # Guardar modelo en outputs (Azure ML sube automáticamente la carpeta outputs/)
    os.makedirs("outputs", exist_ok=True)
    joblib.dump(model, "outputs/model.joblib")

    print("Accuracy:", acc)
    print("Modelo guardado en outputs/model.joblib")

if __name__ == "__main__":
    main()
"""

with open("train.py", "w", encoding="utf-8") as f:
    f.write(train_py)

print("Archivo train.py creado.")


In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

env = Environment("env-mlflow-sklearn")
conda_dep = CondaDependencies()

conda_dep.add_pip_package("mlflow")
conda_dep.add_pip_package("scikit-learn")
conda_dep.add_pip_package("joblib")

env.python.conda_dependencies = conda_dep

print("Environment listo:", env.name)


In [ ]:
from azureml.core import Experiment
from azureml.core.compute import ComputeTarget

experiment_name = "exp-mi-primer-script"
exp = Experiment(ws, experiment_name)

compute_name = "tu-compute-instance-o-cluster"  # <-- cambia esto

compute_target = ComputeTarget(workspace=ws, name=compute_name)
print("Compute:", compute_target.name)


In [ ]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(
    source_directory=".",
    script="train.py",
    compute_target=compute_target,
    environment=env
)

run = exp.submit(src)
print("Run submitted:", run.id)


In [ ]:
run.wait_for_completion(show_output=True)


In [ ]:
metrics = run.get_metrics()
print(metrics)


In [ ]:
import os

os.makedirs("downloaded_outputs", exist_ok=True)
run.download_files(prefix="outputs", output_directory="downloaded_outputs", append_prefix=False)

print("Archivos descargados a downloaded_outputs/")
!ls -la downloaded_outputs/outputs
